part 3 is for exploratory data analysis (EDA) of classes which have less scores than other respective classes & adjusting these classes' images respectively

# Imports

In [2]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)

import pandas as pd
import numpy as np

import string
import logging
import regex as re
import pickle
import os
import glob
import base64

from IPython.display import display
import matplotlib.pyplot  as plt
from PIL import Image, ImageFile, ImageOps, ImageDraw
from pathlib import Path

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from plotly.offline import init_notebook_mode

from tqdm.notebook import tqdm
from collections import Counter
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
from functools import partial

# Global Variables & Functions

In [3]:
dataDir = 'dataset/'
dataReDir = 'dataset_related/'
imgTypes = list(filter(lambda v: re.match('\d{1,3}\. ', v), os.listdir(dataDir)))
imgTypes

['00. selfies',
 '10. fmemes',
 '20. ememes',
 '30. fSocialMedia',
 '40. eSocialMedia',
 '50. fTxtMssgs',
 '70. eGreetingAndMisc',
 '81. academicPhotos',
 '82. academicDigital']

In [4]:
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath):
    with open(fullPath, 'rb') as f:
        content = pickle.load(f)
    return content

def pklForceLoad(path, dtype = 'dict'):
    try:
        content = pklLoad(path)
        return content
    except Exception as e:
        if dtype == 'list':
            pklSave([], path)
            return []
        else:
            pklSave({}, path)
            return {}

# more about naming standards for path components here: https://stackoverflow.com/questions/2235173/what-is-the-naming-standard-for-path-components
def joinPaths(baseDirectory, relativePath):
    return os.path.normpath(os.path.join(baseDirectory, relativePath))

def changeSep(path, newSep, oldSep=os.path.sep): 
    # source: https://stackoverflow.com/questions/18707338/print-raw-string-from-variable-not-getting-the-answers#:~:text=To%20turn%20a%20variable%20to%20raw%20str%2C%20just%20use
    return os.path.normpath(rf"{path}").replace(os.path.sep, newSep) # source: https://mail.python.org/pipermail/tutor/2011-July/084788.html

def pillowOpenOriented(path):
    img = Image.open(path)
    return ImageOps.exif_transpose(img)

## Setting up images and Labels' dictionaries

In [5]:
imgTypeToNum = pklLoad('./dataset_related./imgTypeToNum.pickle')
numToImgType = {}
for  i, type in enumerate(imgTypes):
    imgTypeToNum[type] = i
    numToImgType[i] = type
imgTypeToNum

{'00. selfies': 0,
 '10. fmemes': 1,
 '20. ememes': 2,
 '30. fSocialMedia': 3,
 '40. eSocialMedia': 4,
 '50. fTxtMssgs': 5,
 '70. eGreetingAndMisc': 6,
 '81. academicPhotos': 7,
 '82. academicDigital': 8}